In [ ]:
!ls /home/askyourcorpus/aditya_qc_experiments/QueryCraft-fork/output/evalResultsDB2

In [ ]:
import pandas as pd
import re
import warnings
from sql_metadata import Parser
from matplotlib import pyplot as plt
import json
import logging
import ibm_db

import sqlite3
from tqdm import tqdm
from itertools import product
from collections import defaultdict
import random
from datetime import datetime
import os
from math import ceil
import random
import sys
from datasets import load_dataset
import os



warnings.filterwarnings('ignore')

In [ ]:
query_keywords_list = [
    "TOP","EXISTS","INTERSECT","SELECT","DISTINCT","RANK","AS",
    "WHERE","AND","OR","BETWEEN","LIKE","COUNT","SUM","AVG",
    "MIN","MAX","GROUP BY","ORDER BY","DESC","OFFSET","FETCH",
    "INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION",
    "HAVING","JOIN"
]
aggregate_keywords = ["COUNT","SUM","AVG","MIN","MAX","TOP"]
rank_keywords = ["RANK"]
fillter_keywords = ["GROUP BY","ORDER BY","FILTER","HAVING","EXISTS"]
join_keywords = [
    "JOIN","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION","INTERSECT"
]
orderby_keywords = ["ORDER BY"]
groupby_keywords = ["GROUP BY"]
where_keywords =["WHERE"]
date_keywords =[
    "NOW","GETDATE","CURRENT_TIMESTAMP","DATEDIFF","DATEADD","YEAR","DAY","MONTH"
]
keyword_pattern = re.compile(r'\b(?:' + '|'.join(query_keywords_list) + r')\b', re.IGNORECASE)

def calculate_classification_new(df):
    ## Create new 3 columns 
    df["Expected count"] = df.index
    df["Expected difficulty"] = df.index
    df["Expected classification_new"] = df.index
    for index, row in df.iterrows():
        sql = row["expected query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in orderby_keywords:
                    classification = "ORDER BY"
                elif keyword in groupby_keywords:
                    classification = "GROUP BY"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE/RATIO"
                elif keyword in join_keywords:
                    classification = "JOIN"
                elif keyword in where_keywords:
                    classification = "WHERE"
                elif keyword in date_keywords:
                    classification = "DATE"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Expected difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Expected difficulty'] ="moderate"
        else:
            df.at[index,'Expected difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Expected classification_new'] = classification
        df.at[index,'Expected count'] =count


    ## Create new 3 columns 
    df["Predicted count"] = df.index
    df["Predicted difficulty"] = df.index
    df["Predicted classification_new"] = df.index
    for index, row in df.iterrows():
        sql = row["generated query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in orderby_keywords:
                    classification = "ORDER BY"
                elif keyword in groupby_keywords:
                    classification = "GROUP BY"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE/RATIO"
                elif keyword in join_keywords:
                    classification = "JOIN"
                elif keyword in where_keywords:
                    classification = "WHERE"
                elif keyword in date_keywords:
                    classification = "DATE"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Predicted difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Predicted difficulty'] ="moderate"
        else:
            df.at[index,'Predicted difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Predicted classification_new'] = classification
        df.at[index,'Predicted count'] =count
    return df

def calculate_classification(df):
    ## Create new 3 columns 
    df["Expected count"] = df.index
    df["Expected difficulty"] = df.index
    df["Expected classification"] = df.index
    for index, row in df.iterrows():
        sql = row["expected query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in rank_keywords:
                    classification = "RANK"
                elif keyword in fillter_keywords:
                    classification = "FILTER"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE"
                elif keyword in join_keywords:
                    classification = "JOIN"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Expected difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Expected difficulty'] ="moderate"
        else:
            df.at[index,'Expected difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Expected classification'] = classification
        df.at[index,'Expected count'] =count

        ## Create new 3 columns 
    df["Predicted count"] = df.index
    df["Predicted difficulty"] = df.index
    df["Predicted classification"] = df.index
    for index, row in df.iterrows():
        sql = row["generated query"]
        count =0
        classification =""
        for keyword in query_keywords_list:
            if keyword in sql:
                count=count+1
                if keyword in rank_keywords:
                    classification = "RANK"
                elif keyword in fillter_keywords:
                    classification = "FILTER"
                elif keyword in aggregate_keywords:
                    classification = "AGGREGATE"
                elif keyword in join_keywords:
                    classification = "JOIN"
                ## join  for category 
                ## pre-trained codellama model what type of query are not correct.

        if count < 6:
            df.at[index,'Predicted difficulty'] ="simple"
        elif count > 5 and count < 9:
            df.at[index,'Predicted difficulty'] ="moderate"
        else:
            df.at[index,'Predicted difficulty'] ="challenging"
        if classification == '':
            classification = 'SELECT'

        df.at[index,'Predicted classification'] = classification
        df.at[index,'Predicted count'] =count
    return df

def tokenize_sql_query(sql_query):
    token_list = []
    try:
        for token in Parser(sql_query).tokens:
            token_list.append(str(token.value))
    except:
        pass
    return token_list



def get_category(query):
    query_keywords_list = ["TOP","EXISTS","INTERSECT","SELECT","DISTINCT","TOP","RANK","AS","WHERE","AND","OR","BETWEEN","LIKE","COUNT","SUM","AVG","MIN","MAX","GROUP BY","ORDER BY","DESC","OFFSET","FETCH","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION","HAVING","JOIN"]
    aggregate_keywords = ["COUNT","SUM","AVG","MIN","MAX","TOP"]
    rank_keywords = ["RANK"]
    fillter_keywords = ["GROUP BY","ORDER BY","FILTER","HAVING","EXISTS"]
    join_keywords = ["JOIN","INNER JOIN","LEFT JOIN","RIGHT JOIN","FULL JOIN","UNION","INTERSECT"]

    count =0
    orderby_keywords = ["ORDER BY"]
    groupby_keywords = ["GROUP BY"]
    where_keywords =["WHERE"]
    date_keywords =["NOW","GETDATE","CURRENT_TIMESTAMP","DATEDIFF","DATEADD","YEAR","DAY","MONTH"]
    for keyword in query_keywords_list:
        if isinstance(query,str) and keyword in query:
            count=count+1
            if keyword in orderby_keywords:
                classification = "ORDER BY"
            elif keyword in groupby_keywords:
                classification = "GROUP BY"
            elif keyword in aggregate_keywords:
                classification = "AGGREGATE/RATIO"
            elif keyword in join_keywords:
                classification = "JOIN"
            elif keyword in where_keywords:
                classification = "WHERE"
            elif keyword in date_keywords:
                classification = "DATE"
    if count < 6:
        classification="simple"
    elif count > 5 and count < 9:
        classification="moderate"
    else:
        classification ="challenging"
    if classification == '':
        classification = 'SELECT'
    return classification


In [ ]:
base_path = "../output/evalResults_qc/"
os.makedirs("../output/inference_",exist_ok=True)
os.makedirs("../output/inference_/accumulated_accuracy",exist_ok=True)
model_paths = os.listdir(base_path)


dict_accuracy_overall = {}
dict_latency_overall = {}


for model_path in model_paths:
    
    df = pd.read_csv(base_path+model_path)
    model_id = model_path.split(".csv")[0]

    
    df["predicted_query_toks"] = df["generated query"].apply(tokenize_sql_query).astype(str).replace(","," ")
    df["expected_query_toks"] = df["expected query"].apply(tokenize_sql_query).astype(str).replace(","," ")
    
    df['expected_sql_classification'] = df['expected query'].apply(lambda sql: get_category(sql))
    df['generated_sql_classification'] = df['generated query'].apply(lambda sql: get_category(sql))
    df = calculate_classification(df,)
    df = calculate_classification_new(df)
    df.to_csv("../output/inference_/{}.csv".format(model_id),index=False)

    filtered_df = df
    grouped = df.groupby('generated_sql_classification')
    execution_accuracy = {}
    
    cat_df = pd.DataFrame(columns=["category","Total evaluation records for category","Total correct responses","Execution accuracy %"])
    for name, group in grouped:
        cat_list=[]
        total_true = group['evalScore'].sum()
        total_records = len(group)
        print(total_true,total_records,"##")
        accuracy = total_true / total_records
        execution_accuracy[name] = accuracy
        cat_list.append(name)
        cat_list.append(total_records)
        cat_list.append(total_true)
        cat_list.append(accuracy)
        print(cat_list)
        new_row_df = pd.DataFrame([cat_list], columns=cat_df.columns)
        cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)
    
    
    tot_list = []
    category = "TOTAL"
    total_eval_records = filtered_df.shape[0]
    evalScore_counts = df[df['evalScore'] == True]['evalScore'].value_counts()[0]
    exec_accuracy =  evalScore_counts/total_eval_records
    tot_list.append(category)
    tot_list.append(total_eval_records)
    tot_list.append(evalScore_counts)
    tot_list.append(exec_accuracy)
    new_row_df = pd.DataFrame([tot_list], columns=cat_df.columns)
    cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)
    cat_df['Execution accuracy %'] =round(cat_df['Execution accuracy %']*100,2)
    cat_df['Latency'] = round(df['latency'].mean(),3)
    
    cat_df.to_csv("../output/inference_/accumulated_accuracy/{}.csv".format(model_id),index=False)
    dict_accuracy_overall[model_id] =  cat_df[cat_df["category"]=="TOTAL"]["Execution accuracy %"].values[0]
    dict_latency_overall[model_id] = round(df['latency'].mean(),3)

In [ ]:
dict_accuracy_overall

In [ ]:
dict_latency_overall

In [ ]:
import numpy as np


base_path_inference = "../output/inference_/"
df_selected = pd.read_csv(base_path_inference+model_path)

# filtered_df = df_selected[df_selected['expected_sql_valid']==True]
grouped = df_selected.groupby('Expected classification_new')

execution_accuracy = {}

cat_df = pd.DataFrame(columns=["category","Total evaluation records for category","Total correct responses","Execution accuracy %"])
for name, group in grouped:
    cat_list=[]
    total_true = group['evalScorePostProcessing'].sum()
    total_records = len(group)
    #print(total_true,total_records,"##")
    accuracy = total_true / total_records
    execution_accuracy[name] = accuracy
    cat_list.append(name)
    cat_list.append(total_records)
    cat_list.append(total_true)
    cat_list.append(accuracy)
    #print(cat_list)
    new_row_df = pd.DataFrame([cat_list], columns=cat_df.columns)
    cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)

total_aggregate = cat_df[cat_df["category"] ==  "TOTAL"]

total_evaluation_records = cat_df['Total evaluation records for category'].sum()
cat_df['Weights'] = cat_df['Total evaluation records for category'] / total_evaluation_records

tot_list = []
category = "TOTAL"
total_eval_records = df_selected.shape[0]
evalScore_counts = df_selected[df_selected['evalScorePostProcessing'] == True]['evalScorePostProcessing'].value_counts()[0]
exec_accuracy =  evalScore_counts/total_eval_records*100
tot_list.append(category)
tot_list.append(total_eval_records)
tot_list.append(evalScore_counts)
tot_list.append(exec_accuracy)
cat_df['Execution accuracy %'] = round(cat_df['Execution accuracy %']*100,2)
cat_df['Weighted execution accuracy'] = cat_df['Execution accuracy %'] * cat_df['Weights']

tot_list.append(np.nan)
tot_list.append(np.nan)
new_row_df = pd.DataFrame([tot_list], columns=cat_df.columns)
cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)
cat_df['Average Latency'] = np.nan
cat_df['Average Latency'].iloc[-1] = round(df_selected['latency'].mean(),3)


In [ ]:
dict(zip(cat_df["category"],cat_df['Execution accuracy %']))

In [ ]:
AGGREGATE_RATIO = []
GROUP_BY = []
ORDER_BY = []
SELECT =[]
WHERE = []
TOTAL_AVERAGE = []

AGGREGATE_RATIO_COUNT = []
GROUP_BY_COUNT = []
ORDER_BY_COUNT = []
SELECT_COUNT =[]
WHERE_COUNT = []
TOTAL_CORRECT_COUNT = []


MODEL_ID_LIST = []

model_results_dir = os.listdir("../output/inference_/")
model_results_dir.remove("accumulated_accuracy")
for model_path in model_results_dir:


    df_selected = pd.read_csv(base_path_inference+model_path)
    grouped = df_selected.groupby('Expected classification_new')

    execution_accuracy = {}

    cat_df = pd.DataFrame(columns=["category","Total evaluation records for category","Total correct responses","Execution accuracy %"])
    for name, group in grouped:
        cat_list=[]
        total_true = group['evalScorePostProcessing'].sum()
        total_records = len(group)
        #print(total_true,total_records,"##")
        accuracy = total_true / total_records
        execution_accuracy[name] = accuracy
        cat_list.append(name)
        cat_list.append(total_records)
        cat_list.append(total_true)
        cat_list.append(accuracy)
        #print(cat_list)
        new_row_df = pd.DataFrame([cat_list], columns=cat_df.columns)
        cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)

    total_aggregate = cat_df[cat_df["category"] ==  "TOTAL"]

    total_evaluation_records = cat_df['Total evaluation records for category'].sum()
    cat_df['Weights'] = cat_df['Total evaluation records for category'] / total_evaluation_records

    tot_list = []
    category = "TOTAL"
    total_eval_records = df_selected.shape[0]
    evalScore_counts = df_selected[df_selected['evalScorePostProcessing'] == True]['evalScorePostProcessing'].value_counts()[0]
    exec_accuracy =  evalScore_counts/total_eval_records*100
    tot_list.append(category)
    tot_list.append(total_eval_records)
    tot_list.append(evalScore_counts)
    tot_list.append(exec_accuracy)
    cat_df['Execution accuracy %'] = round(cat_df['Execution accuracy %']*100,2)
    cat_df['Weighted execution accuracy'] = cat_df['Execution accuracy %'] * cat_df['Weights']

    tot_list.append(np.nan)
    tot_list.append(np.nan)
    new_row_df = pd.DataFrame([tot_list], columns=cat_df.columns)
    cat_df = pd.concat([cat_df, new_row_df], ignore_index=True)
    cat_df['Average Latency'] = np.nan
    cat_df['Average Latency'].iloc[-1] = round(df_selected['latency'].mean(),3)

    model_results_dict = dict(zip(cat_df["category"],cat_df['Execution accuracy %']))

    model_count_dict = dict(zip(cat_df["category"],cat_df['Total correct responses']))


    AGGREGATE_RATIO.append(model_results_dict['AGGREGATE/RATIO'])
    GROUP_BY.append(model_results_dict['GROUP BY'])
    ORDER_BY.append(model_results_dict['ORDER BY'])
    SELECT.append(model_results_dict['SELECT'])
    WHERE.append(model_results_dict['WHERE'])
    TOTAL_AVERAGE.append(model_results_dict['TOTAL'])



    AGGREGATE_RATIO_COUNT.append(model_count_dict['AGGREGATE/RATIO'])
    GROUP_BY_COUNT.append(model_count_dict['GROUP BY'])
    ORDER_BY_COUNT.append(model_count_dict['ORDER BY'])
    SELECT_COUNT.append(model_count_dict['SELECT'])
    WHERE_COUNT.append(model_count_dict['WHERE'])




    MODEL_ID_LIST.append(model_path.split(".csv")[0].replace("_exEvaluator",""))


df_accuracy_query_type = pd.DataFrame({"Model Id":MODEL_ID_LIST,'SELECT': SELECT,'ORDER BY': ORDER_BY,'WHERE': WHERE, 'GROUP BY': GROUP_BY,"AGGREGATE/RATIO":AGGREGATE_RATIO,'TOTAL AVERAGE ACCURACY': TOTAL_AVERAGE,})

df_count_query_type = pd.DataFrame({"Model Id":MODEL_ID_LIST,'SELECT': SELECT_COUNT,'ORDER BY': ORDER_BY_COUNT,'WHERE': WHERE_COUNT, 'GROUP BY': GROUP_BY_COUNT,"AGGREGATE/RATIO":AGGREGATE_RATIO_COUNT,})


# tolal_list = ["Total evaluation records for category"]
# tolal_list.extend(cat_df["Total evaluation records for category"][:-1].to_list())
# new_row_df = pd.DataFrame([tolal_list], columns=df_count_query_type.columns)
# df_count_query_type = pd.concat([df_count_query_type, new_row_df], ignore_index=True)


In [ ]:
df_accuracy_query_type

In [ ]:
df_count_query_type